In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
import missingno as mns 
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import json


In [3]:

# Load the JSON file
json_file_path = "IMDB_250_movies.json"

with open(json_file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Convert list values to plain values
for movie in data:
    for key, value in movie.items():
        if isinstance(value, list):
            if all(isinstance(i, list) for i in value):  # If nested lists exist, flatten them
                movie[key] = ", ".join([", ".join(map(str, sublist)) for sublist in value])
            else:
                movie[key] = ", ".join(map(str, value))

# Convert to DataFrame
df = pd.DataFrame(data)

# Save as CSV
csv_file_path = "IMDB_250_movies_cleaned.csv"
df.to_csv(csv_file_path, index=False, encoding="utf-8")

csv_file_path


'IMDB_250_movies_cleaned.csv'

In [5]:
## reading the csv dataset 
df = pd.read_csv("IMDB_250_movies_cleaned.csv")
df.head(6)

,title,movie_rank,genres,release_day,release_month,release_year,movie_length,rating,vote_count,description
0,The Shawshank Redemption,1,Drama,14,10,1994,2:22:00,9.3,3020598,A banker convicted of uxoricide forms a friend...
1,The Godfather,2,"Crime, Drama",24,3,1972,2:55:00,9.2,2108019,The aging patriarch of an organized crime dyna...
2,The Dark Knight,3,"Action, Crime, Drama",18,7,2008,2:32:00,9.0,2996913,When a menace known as the Joker wreaks havoc ...
3,The Godfather Part II,4,"Crime, Drama",18,12,1974,3:22:00,9.0,1419714,The early life and career of Vito Corleone in ...
4,12 Angry Men,5,"Crime, Drama",10,4,1957,1:36:00,9.0,916939,The jury in a New York City murder trial is fr...
5,The Lord of the Rings: The Return of the King,6,"Adventure, Drama, Fantasy",17,12,2003,3:21:00,9.0,2063799,Gandalf and Aragorn lead the World of Men agai...
